In [97]:
from sklearn import datasets, model_selection,metrics
import pandas as pd
import numpy as np

In [2]:
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = pd.Series(iris.target)

In [3]:
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
y = df_iris['target'].values
df_iris.pop('target')
X = df_iris.values

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [6]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [7]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [8]:
def mse(y_pred, y_true):
    return ((y_true - y_pred) ** 2).mean()

In [161]:
class Perceptron:
    def __init__(self, n_inputs, n_outputs, n_layers, n_neurons, activation=None, max_iter=1000, eta=0.1):
        # количество входящих нейронов
        self.n_inputs = n_inputs
        # количество скрытых слоев
        self.n_outputs = n_outputs
        self.n_layers = n_layers
        # количество нейронов на скрытом слое
        self.n_neurons = n_neurons
        # создание скрытых слоев
        self.layers = []
        self.activation = activation
        self.max_iter = max_iter
        self.eta = eta
        
    def create_layers(self):
        n_inputs = self.n_inputs
        # создание скрытых слоев
        for i in range(self.n_layers):
            self.layers.append(self.NeuronLayer(n_inputs, self.n_neurons))
            n_inputs = self.n_neurons
        # создание выходного слоя
        self.layers.append(self.NeuronLayer(n_inputs, self.n_outputs))
        
    def back_propogation(self, error, values, X):
        weights = 1
        for i in reversed(range(len(self.layers))):
            self.layers[i].delta = np.dot(error, np.transpose(weights))
            derivative = np.transpose(sigmoid_derivative(values))
            for j in range(self.layers[i].delta.shape[1]):
                self.layers[i].delta[0][j] *= derivative[0][j]
            if i == 0:
                values = X
            else:
                values = self.layers[i - 1].values
            values = values.reshape(-1, 1)
            self.layers[i].weights -= self.eta * np.dot(values, self.layers[i].delta)
            weights = self.layers[i].weights
            error = self.layers[i].delta
            
    def get_weights(self):
        weights = []
        for layer in self.layers:
            weights.append(layer.weights)
        return weights
            
    def forward_propogation(self, x):
        inputs = x
        for layer in self.layers:
            if self.activation is not None:
                layer.values = self.activation(np.dot(inputs, layer.weights) + layer.b)
            else:
                layer.values = np.dot(inputs, layer.weights) + layer.b
            inputs = layer.values
        return inputs
    
    def fit(self, X, y):
        self.create_layers()
        for i in range(self.max_iter):
            index = np.random.randint(0, X.shape[0])
            inputs = X[index].reshape(1, -1)
            outputs = self.forward_propogation(inputs)
            error = outputs - y[index]
            self.back_propogation(error, outputs, X[index])
            
    def predict(self, X):
        outputs = np.ones((X.shape[0], 1))
        for i in range(X.shape[0]):
            outputs[i] = self.forward_propogation(X[i].reshape(1, -1))
        return outputs
            
    class NeuronLayer:
        def __init__(self, n_inputs, n_neurons):
            self.weights = np.random.normal(loc=0, scale=0.1, size=(n_inputs, n_neurons))
            self.values = []
            self.delta = []
            self.b = np.ones((1, n_neurons))

In [163]:
perceptron = Perceptron(4, 1, 2, 2, sigmoid)
perceptron.fit(X_train, y_train)

[[-0.27367871]]
[[0.19877867]]
[[ 0.00588335 -0.00462878]]
[[0.19103405 0.19613732]]
[[ 1.19511196e-04 -8.25194236e-05]]
[[0.14914828 0.24303401]]
[[-0.27246771]]
[[0.19822906]]
[[ 0.00562459 -0.00480953]]
[[0.1913149  0.19547766]]
[[ 1.16351706e-04 -8.41187951e-05]]
[[0.15954647 0.24988095]]
[[0.72873712]]
[[0.19767933]]
[[-0.01654093  0.01130216]]
[[0.19157386 0.194693  ]]
[[-0.0003271   0.00020664]]
[[0.16519121 0.23381968]]
[[-1.27445607]]
[[0.19912994]]
[[ 0.02435625 -0.02462415]]
[[0.19107131 0.19626454]]
[[ 0.00052414 -0.00041816]]
[[0.15584665 0.23850962]]
[[-1.26899312]]
[[0.19663582]]
[[ 0.0193238  -0.02876861]]
[[0.19110714 0.19622763]]
[[ 0.00046511 -0.0004616 ]]
[[0.15590911 0.23855036]]
[[-0.26363901]]
[[0.19413348]]
[[ 0.00376914 -0.00609298]]
[[0.19141039 0.19532076]]
[[ 9.32188651e-05 -9.64673234e-05]]
[[0.16197071 0.24948269]]
[[-0.26259171]]
[[0.1936373]]
[[ 0.00355258 -0.0062427 ]]
[[0.19116497 0.1958517 ]]
[[ 9.05245657e-05 -9.81630757e-05]]
[[0.15119818 0.24712442

[[-0.17670602]]
[[0.145481]]
[[-0.00703855 -0.01178785]]
[[0.19056645 0.19377414]]
[[-0.00011133 -0.00014676]]
[[0.16013067 0.24966894]]
[[0.82383923]]
[[0.14512815]]
[[0.03167291 0.05376707]]
[[0.19077382 0.19268291]]
[[0.00048967 0.00065702]]
[[0.16018292 0.22990858]]
[[-1.17831384]]
[[0.14651801]]
[[-0.04795373 -0.07982914]]
[[0.19022274 0.19470953]]
[[-0.00076654 -0.00099638]]
[[0.14483888 0.22848303]]
[[-0.17541241]]
[[0.14464289]]
[[-0.00709527 -0.01177935]]
[[0.19051566 0.19348503]]
[[-0.00011427 -0.00014625]]
[[0.15489409 0.24965398]]
[[-1.17511836]]
[[0.14445192]]
[[-0.04961399 -0.08092831]]
[[0.19037901 0.19433077]]
[[-0.00082627 -0.00102057]]
[[0.15857431 0.23767401]]
[[-0.17234918]]
[[0.14264494]]
[[-0.00723062 -0.01176547]]
[[0.19031177 0.19367914]]
[[-0.00012099 -0.0001481 ]]
[[0.15112845 0.24904165]]
[[-0.17195657]]
[[0.14238751]]
[[-0.00724573 -0.0117617 ]]
[[0.19043366 0.1935504 ]]
[[-0.00012191 -0.00014823]]
[[0.15917578 0.24977061]]
[[-0.17156195]]
[[0.14212845]]
[[-

[[0.00085297 0.00058593]]
[[0.15624586 0.23256243]]
[[-1.15728331]]
[[0.13254527]]
[[-0.05815193 -0.08590147]]
[[0.19013781 0.19349165]]
[[-0.00124646 -0.00084948]]
[[0.15770055 0.24683772]]
[[0.84509779]]
[[0.13090751]]
[[0.04102987 0.06104764]]
[[0.19051521 0.19219811]]
[[0.00086528 0.00058669]]
[[0.16398356 0.23002866]]
[[-0.15661758]]
[[0.13208851]]
[[-0.00770433 -0.0114473 ]]
[[0.1900517  0.19324732]]
[[-0.00016277 -0.00011115]]
[[0.14873049 0.24954733]]
[[-0.15634405]]
[[0.13190059]]
[[-0.00771161 -0.01144243]]
[[0.18996299 0.19347984]]
[[-0.00016343 -0.00011155]]
[[0.14904333 0.24590729]]
[[0.84409116]]
[[0.13160127]]
[[0.04062252 0.06072284]]
[[0.19068664 0.19210897]]
[[0.00084843 0.00057408]]
[[0.16821764 0.2211176 ]]
[[-1.15769666]]
[[0.13282842]]
[[-0.05799593 -0.08580343]]
[[0.19003538 0.19366141]]
[[-0.00123958 -0.0008347 ]]
[[0.15237383 0.24353466]]
[[-1.15544237]]
[[0.13128004]]
[[-0.05892245 -0.08633518]]
[[0.18996777 0.19355401]]
[[-0.00129029 -0.00085401]]
[[0.1539907

[[-1.14062327]]
[[0.12084836]]
[[-0.06443443 -0.08893159]]
[[0.18906902 0.19329994]]
[[-0.00170914 -0.00064131]]
[[0.13351646 0.22443723]]
[[-0.13872048]]
[[0.11947711]]
[[-0.00776797 -0.01071329]]
[[0.18943967 0.19256812]]
[[-2.06682658e-04 -7.70140060e-05]]
[[0.14432399 0.24868071]]
[[0.86156205]]
[[0.11927288]]
[[0.04737652 0.06564094]]
[[0.19022846 0.19160683]]
[[0.00124553 0.00045266]]
[[0.1642924  0.22451475]]
[[0.8601822]]
[[0.12026878]]
[[0.0468987  0.06528963]]
[[0.19018251 0.19177032]]
[[0.00121525 0.0004367 ]]
[[0.15803443 0.22816449]]
[[-1.14130733]]
[[0.12133957]]
[[-0.06421033 -0.08881627]]
[[0.18962446 0.19285546]]
[[-0.00169583 -0.00061801]]
[[0.14887063 0.24749721]]
[[-1.1394313]]
[[0.11999021]]
[[-0.06478622 -0.08906623]]
[[0.18952342 0.19271579]]
[[-0.00174421 -0.0006373 ]]
[[0.14869322 0.24752525]]
[[0.8624915]]
[[0.11859991]]
[[0.0476923  0.06586133]]
[[0.19005711 0.19156908]]
[[0.00126695 0.00045172]]
[[0.15755341 0.22690021]]
[[0.86110144]]
[[0.11960575]]
[[0.047

[[-0.0025758  -0.00059103]]
[[0.13399515 0.24268966]]
[[0.88877115]]
[[0.09885699]]
[[0.0551991  0.07052299]]
[[0.18861826 0.18955479]]
[[0.0019965  0.00043905]]
[[0.15749589 0.22412067]]
[[0.88777063]]
[[0.09963394]]
[[0.0549853  0.07041394]]
[[0.18864292 0.1897076 ]]
[[0.00196786 0.00042091]]
[[0.15442895 0.22665519]]
[[-1.11332906]]
[[0.10048558]]
[[-0.07048206 -0.08998935]]
[[0.18814065 0.19078713]]
[[-0.00255858 -0.00056438]]
[[0.14970346 0.24796864]]
[[-1.11204616]]
[[0.09949182]]
[[-0.0706214 -0.089907 ]]
[[0.18802809 0.19064482]]
[[-0.00259794 -0.00058374]]
[[0.14945748 0.24798978]]
[[0.88929675]]
[[0.09844804]]
[[0.05530969 0.07057207]]
[[0.18849981 0.18952984]]
[[0.0020123  0.00043504]]
[[0.15386561 0.22655394]]
[[0.88828625]]
[[0.09923379]]
[[0.0551069  0.07047512]]
[[0.18869718 0.18965338]]
[[0.00198523 0.00041631]]
[[0.15752347 0.22411389]]
[[-1.11278535]]
[[0.10006482]]
[[-0.07054146 -0.08994726]]
[[0.18782925 0.19088715]]
[[-0.00257542 -0.00055728]]
[[0.14046915 0.242477

[[0.18686415 0.18879122]]
[[-2.71986836e-04 -4.47549641e-05]]
[[0.14213955 0.24985807]]
[[-0.0973381]]
[[0.08786339]]
[[-0.00627068 -0.00774549]]
[[0.18654345 0.18882611]]
[[-2.71598035e-04 -4.49160622e-05]]
[[0.1309069  0.24961507]]
[[-0.0972837]]
[[0.08781958]]
[[-0.00626954 -0.00774276]]
[[0.18678232 0.18882281]]
[[-2.72037506e-04 -4.50258851e-05]]
[[0.1409651  0.24999601]]
[[0.90283207]]
[[0.08772632]]
[[0.05765152 0.07130992]]
[[0.18743142 0.18810617]]
[[0.00248071 0.00039176]]
[[0.15180434 0.22393834]]
[[-0.09803469]]
[[0.08842389]]
[[-0.00631555 -0.00781043]]
[[0.18711961 0.1888757 ]]
[[-2.72158969e-04 -4.34623061e-05]]
[[0.14902607 0.24928923]]
[[-0.09791356]]
[[0.0883265]]
[[-0.00630642 -0.00779778]]
[[0.18706416 0.18870138]]
[[-2.71897135e-04 -4.35201319e-05]]
[[0.1439908  0.24757155]]
[[-0.09781662]]
[[0.08824853]]
[[-0.00630021 -0.00778875]]
[[0.18673782 0.18889154]]
[[-2.71714844e-04 -4.37751860e-05]]
[[0.136079   0.24993731]]
[[-1.09770369]]
[[0.08815768]]
[[-0.07133318 -

[[-2.80320387e-04 -1.85454395e-05]]
[[0.12952494 0.24992278]]
[[-1.09209266]]
[[0.0836116]]
[[-0.07125761 -0.08683977]]
[[0.18627276 0.18845999]]
[[-0.00338524 -0.00024189]]
[[0.13210989 0.24942738]]
[[0.9087605]]
[[0.08291485]]
[[0.05837511 0.07123407]]
[[0.18716573 0.18775264]]
[[0.00275422 0.00017509]]
[[0.14952063 0.223553  ]]
[[0.90803077]]
[[0.08351089]]
[[0.05831592 0.0712577 ]]
[[0.18719994 0.18787817]]
[[0.0027294  0.00015637]]
[[0.14675041 0.22512341]]
[[-0.09267213]]
[[0.08408401]]
[[-0.00599704 -0.00732689]]
[[0.18636278 0.18848101]]
[[-2.80522706e-04 -1.66010356e-05]]
[[0.12957848 0.24992319]]
[[0.90734621]]
[[0.08406906]]
[[0.05826964 0.07128852]]
[[0.18740352 0.18798659]]
[[0.00270915 0.00013832]]
[[0.14936692 0.22115578]]
[[0.90656606]]
[[0.08470404]]
[[0.058217   0.07132388]]
[[0.18748485 0.18827843]]
[[0.00268621 0.00012047]]
[[0.15257258 0.2303349 ]]
[[-1.09415575]]
[[0.08529045]]
[[-0.0714046 -0.0873291]]
[[0.18661505 0.1889108 ]]
[[-0.00332501 -0.00017205]]
[[0.137

In [164]:
y_pred = perceptron.predict(X_test)